In [1]:
!pip install opencv-python matplotlib imageio tensorflow

In [4]:
import os
import cv2
import tensorflow as tf
import numpy as np
from typing import List
from matplotlib import pyplot as plt
import imageio

2024-03-11 13:01:15.631761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 13:01:15.948829: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 13:01:16.023499: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mathildaweston/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/cv2/../../

In [30]:
speakers = [f's{i}' for i in [i for i in range(1, 35) if i != 21]]

# paths_to_clean = ['/'.join([os.getcwd(),'data/alignments', speaker]) for speaker in speakers]
# path_to_clean

In [32]:
# speakers

In [5]:
# speakers = 's1'

# path_to_clean = ['/'.join([os.getcwd(),'data/alignments', speaker])]
# path_to_clean

In [6]:
# file_names = []
# for folder_path in paths_to_clean:
#     for root, dirs, files in os.walk(folder_path):
#         for file in files:
#             if file.endswith('.align'):
#                 file_name = file.replace(".align", "")
#                 file_names.append(file_name)
# len(file_names)

In [5]:
vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]

char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

2024-03-11 13:01:24.644890: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mathildaweston/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/cv2/../../lib64:
2024-03-11 13:01:24.645541: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-11 13:01:24.645565: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-12451Q9): /proc/driver/nvidia/version does not exist
2024-03-11 13:01:24.647089: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow wit

In [6]:
def load_alignments(path:str) -> List[str]: 
    with open(path, 'r') as f: 
        lines = f.readlines() 
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil': #Removing the silences
            tokens = [*tokens,' ',line[2]]
    return char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

In [7]:
def load_align_data(file_name: str): 
    alignment_path = os.path.join(path_to_clean[0], f'{file_name}.align')
    alignments = load_alignments(alignment_path).numpy().tolist()

    return alignments

In [8]:
# len(load_align_data('srah7a'))

In [9]:
def make_dict(file_names):
    align_dict = dict(zip(file_names, [load_align_data(file_name) for file_name in file_names]))
    return align_dict

In [12]:
def preproc_align(speaker: str):
    path_to_clean = ['/'.join([os.getcwd(),'raw_alignments', speaker, 'align'])]
    file_names = []
    for folder_path in path_to_clean:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith('.align'):
                    file_name = file.replace(".align", "")
                    file_names.append(file_name)
    def load_alignments(path:str) -> List[str]: 
        with open(path, 'r') as f: 
            lines = f.readlines() 
        tokens = []
        for line in lines:
            line = line.split()
            if line[2] != 'sil': #Removing the silences
                tokens = [*tokens,' ',line[2]]
        return char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

    def load_align_data(file_name: str): 
        alignment_path = os.path.join(path_to_clean[0], f'{file_name}.align')
        alignments = load_alignments(alignment_path).numpy().tolist()

        return alignments

    def make_dict(file_names):
        align_dict = dict(zip(file_names, [load_align_data(file_name) for file_name in file_names]))
        return align_dict
    
    return make_dict(file_names)

In [18]:
s2_dict = preproc_align(speakers[0])

In [20]:
s2_dict.keys()

dict_keys(['bgwh9p', 'lwar5n', 'pwij4a', 'bbws9p', 'pbwj1n', 'brwt4s', 'sgwx4a', 'srih1p', 'pwad3p', 'pwbd7p', 'bwbn2s', 'lbbk5p', 'lwws5p', 'pwbq3n', 'lbwlza', 'pbio3n', 'swbczs', 'pwwq8s', 'pbio6a', 'srbo2s', 'bbbl9n', 'lrbr3n', 'swio1p', 'swai1p', 'pbbi8s', 'brbg1n', 'srbb5p', 'sgwp7n', 'pbbp1n', 'bbie9p', 'bwam9p', 'pbio5p', 'sgwd1p', 'lgil5p', 'bbwm6a', 'pgiq2s', 'sbbu1p', 'pgikza', 'lrbr5p', 'bwbn4a', 'swiu3n', 'sbwh8a', 'lwiy8a', 'lbiq2a', 'bbwm4s', 'bbbs6a', 'prip2a', 'pbih9n', 'pwaj8a', 'swbc2a', 'srwb9p', 'bgit6a', 'bram3p', 'srbh8s', 'lrak6s', 'lbbe2a', 'bgbn9p', 'sgwc9n', 'swih6s', 'lrbe5n', 'lgir8s', 'pwiv9n', 'brwa2s', 'pwip7p', 'swwv5n', 'srau2s', 'pwbyza', 'bwwu1p', 'sgii1n', 'sbba7n', 'pric1n', 'lbij8a', 'sbag9p', 'lbwk8s', 'bris4a', 'sbag7n', 'lwbf4a', 'pgak1n', 'pbac2a', 'swbo8s', 'lgws9n', 'lbad8a', 'sbwu6a', 'sgwj5p', 'bbwgzs', 'sgai6s', 'pwax4s', 'brif5p', 'lbax8s', 'bbwg2a', 'bwwn7p', 'lrbl1p', 'bwbt5n', 'bwag3n', 'lrae1n', 'pwax6a', 'lgwaza', 'briz5n', 'lbbk6a',

### Saving to json

In [37]:
import json

def save_json(speaker: str):
    with open('/'.join([os.getcwd(), 'data/processed_alignments', f'{speaker}_proc_align.json']), 'w') as fp:
        json.dump(preproc_align(speaker), fp)
    
    
    
    
# with open('/'.join([os.getcwd(), 'data/processed_alignments', speaker, f'{speaker}_proc_align.json']), 'w') as fp:
#     json.dump(s1_aligns, fp)

In [ ]:
# f = open('/'.join([os.getcwd(), 'data/processed_alignments', speaker, f'{speaker}_proc_align.json']))

# test_align = json.load(f)

In [ ]:
# test_align

### Run and save for all speakers

In [38]:
for speaker in speakers:
    save_json(speaker)

In [47]:
f = open('/'.join([os.getcwd(), 'data/processed_alignments', f's22_proc_align.json']))

test_align = json.load(f)

In [48]:
len(test_align.keys())

1000